#  Predict which water pumps are faulty

**The goal is to identify with above 60% accuracy which water wells are faulty or non functional.** I will be using data from Taarifa and the Tanzanian Ministry of Water. The submission of my predictions will be in the format of .CSV with columns for 'id' as well as 'status_group'. Lets start by loading the data and getting a feel for it. 

In [1]:
import pandas as pd
import zipfile
zf_labl = zipfile.ZipFile('C:/Users/dakot/Downloads/train_labels.csv.zip')
zf_content = zipfile.ZipFile('C:/Users/dakot/Downloads/train_features.csv.zip')
df_label = pd.read_csv(zf_labl.open(zipfile.ZipFile.namelist(zf_labl)[0])) 
df_feats = pd.read_csv(zf_content.open(zipfile.ZipFile.namelist(zf_content)[0]))

In [2]:
df_label.describe(include='object')

,status_group
count,59400
unique,3
top,functional
freq,32259


**The df_label data frame contains the id along with status of the well. The status will be our dependant variable for this project, I believe I will utilize the ID label with the other features to ensure tracability throughout.**

In [3]:
#let us see the whole column profile of the data frame 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df_feats.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
df_feats.shape, df_label.shape

((59400, 40), (59400, 2))

**The df_feats Data Frame contains all the features we will use to predict the status of any given well.** For the first iteration I will run a simple baseline. We see the shape of the features df is 59400 by 40 and the shape of the label df is 59400 by 2. Lets check the distribution of the status of the wells. 

In [5]:
df_label.status_group.value_counts(normalize = True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

**Overall it appears the majority of wells are functional(54.3%), non functional is the second highest(38.42%). Functional needing repair rounds out the data set(7.26%).** If I were to make a blind prediction saying that all the wells were functional I would be correct around 54 pearcent of the time. Not bad but not nearly conclusive or useful for the real world. Lets dig deeper. 

For simplicity I will combine the two data frames to process before splitting. 


In [6]:
full = pd.DataFrame.merge(df_label,df_feats)

In [7]:
full.head()

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [8]:
full.isnull().sum()

id                           0
status_group                 0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_qu

**After merging the data frames together I find there are some Null values that may skew our results or otherwise break our functions during the process.** For now we will drop all instances that are missing values, but later we may impute some values to help our models predict better if neeeded.  

In [9]:
clean = full.dropna()

In [10]:
clean.isna().sum()

id                       0
status_group             0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
s

**Now that we have no NaN values lets make some test and tarin sets with our data. We want to predict status so we will call that the 'y' variable. All other features will be called our 'X' matrix of features.** 

In [11]:
from sklearn.model_selection import train_test_split
X1 = clean.drop(columns = ['status_group',], axis = 1)
y = clean['status_group']
X_train, X_test, y_train, y_test = train_test_split(X1, y, random_state=42)

In [12]:
X_train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
13690,22803,0.0,2011-03-19,Government Of Tanzania,1091,DWE,38.386908,-5.026557,Selepta,0,Pangani,Mission,Tanga,4,2,Korogwe,Bungu,150,True,GeoData Consultants Ltd,VWC,Gale water supply,False,1972,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,dry,dry,spring,spring,groundwater,communal standpipe multiple,communal standpipe
3474,64841,0.0,2011-04-17,World Bank,0,DWE,35.903096,-4.524686,Zahanati,0,Internal,Atta Shule,Dodoma,1,1,Kondoa,Kisese,0,True,GeoData Consultants Ltd,VWC,Atta,False,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
25853,42253,0.0,2012-11-03,Government Of Tanzania,0,Central government,32.970934,-4.925031,Barazani,0,Lake Tanganyika,Upuge A,Tabora,14,3,Uyui,Upuge,0,True,GeoData Consultants Ltd,VWC,Upuge water supply,True,0,mono,mono,motorpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
15676,51862,300.0,2013-03-08,District Council,1278,District Council,37.658215,-3.665646,Kwa Jabiri,0,Pangani,Mwira B,Kilimanjaro,3,2,Mwanga,Shinghatini,43,True,GeoData Consultants Ltd,VWC,Churu water supply,False,2007,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
11105,65269,0.0,2013-01-24,Ministry Of Water,1112,Idara ya maji,34.961933,-5.841291,Sekondari Kilimatinde,0,Internal,Mabwenini,Singida,13,3,Manyoni,Kilimatinde,400,True,GeoData Consultants Ltd,VWC,Kilimatinde water supply,False,1998,mono,mono,motorpump,vwc,user-group,unknown,unknown,soft,good,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe


In [13]:
X_train.isna().sum().sum()

0

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import category_encoders as ce
import numpy as np 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df
      

In [15]:
X_train_DC = dummyEncode(X_train)
X_train_DC.head()
X_test_DC = dummyEncode(X_test)
X_test_DC.head()

C:\Users\dakot\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
3582,28203,0.0,227,183,1328,220,37.995135,-4.256169,4842,0,5,4011,6,3,3,87,1105,500,1,0,5,1581,1,2002,3,1,0,6,4,0,2,5,2,0,0,8,6,0,1,1
11449,65922,0.0,51,191,1211,227,36.867741,-6.056682,3959,0,8,1901,11,5,1,32,971,250,1,0,5,881,1,2005,13,10,5,6,4,0,2,5,2,1,1,3,0,0,1,1
25618,71108,5.0,276,128,873,45,37.135462,-3.435867,2919,0,5,3926,6,3,5,12,591,350,1,0,8,1555,1,1999,3,1,0,8,4,4,5,5,2,1,1,8,6,0,1,1
11009,39550,0.0,65,462,0,255,36.237615,-6.163544,184,0,8,1110,2,1,1,64,91,0,1,0,5,1388,1,0,9,6,3,6,4,4,5,3,4,1,1,7,5,0,6,5
16941,1393,0.0,143,131,0,149,34.290564,-3.886278,4368,0,0,2912,17,17,6,54,795,0,1,0,2,1251,1,0,3,1,0,2,2,0,2,5,2,3,3,5,3,1,1,1


In [16]:
X_train_DC.isna().sum().sum()

0

In [17]:
model= LogisticRegression()
model.fit(X_train_DC, y_train)
y_pred = model.predict(X_test_DC)
accuracy_score(y_test, y_pred)


C:\Users\dakot\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\dakot\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6709807305148117

In [20]:
pipeline = make_pipeline(ce.OneHotEncoder(use_cat_names=True),
                         StandardScaler(), LogisticRegression(solver ='lbfgs',n_jobs=-1, multi_class = 'auto',C=2))
pipeline.fit(X_train_DC, y_train)

C:\Users\dakot\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\dakot\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


Pipeline(memory=None,
     steps=[('onehotencoder', OneHotEncoder(cols=[], drop_invariant=False, handle_unknown='impute',
       impute_missing=True, return_df=True, use_cat_names=True, verbose=0)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logisticregression', LogisticRegression(C=2, cla...enalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False))])

In [21]:
y_pred = pipeline.predict(X_test_DC)
accuracy_score(y_test,y_pred)

C:\Users\dakot\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.6711245326430831

In [23]:
pred = pd.DataFrame(y_pred)

In [25]:
pred.shape

(6954, 1)